In [31]:
import pandas as pd
import numpy as np
import os, sys, regex

In [32]:
dala = os.getenv('HOME')

In [33]:
base_dir = f'{dala}/Documents/RunSheets/AllRunsheets'

In [34]:
cols1 = ['Sample_Name', 'Reporter', 'CT', 'Ct_Mean']

data = pd.DataFrame(columns=cols1)
data_list = []
for file in os.listdir(base_dir):
    if file.endswith('.xls'):
        data_list.append(file)
    pass
sheets = sorted(data_list)

In [35]:
len(np.unique(np.array(sheets)))

428

In [36]:
' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2])

'2020-10-29 15:46:45'

In [37]:
pd.to_datetime(' '.join('2020-10-29 15:46:45 PM EAT'.split(' ')[:-2]), yearfirst=True)

Timestamp('2020-10-29 15:46:45')

In [38]:
cols2 = ['Sample Name', 'Reporter', 'CT', 'Ct Mean', 'Cт', 'Cт Mean']

counts = 0

for sheet in sheets:
    df = pd.read_excel(f'{base_dir}/{sheet}')
    count2 = 0
#     print(sheet)
    for row in df.itertuples():
        count2 += 1
        if row[1] == 'Experiment Run End Time':
            expt_run_date = pd.to_datetime(' '.join(row[2].split(' ')[:-2]), yearfirst=True)
        elif row[1] == 'Well':
            df = pd.read_excel(f'{base_dir}/{sheet}', 'Results', header=count2)
            cols3 = list(set(df.columns) & set(cols2))
#             print(cols3)
#             print(list(df.columns))
            df2 = df[sorted(cols3)]
            df2.columns = sorted(cols1)
            df3 = df2.assign(Dt_Run=expt_run_date)
            data = data.append(df3, ignore_index=True)
            counts += 1
            break
        else: pass

In [39]:
print(f'number of sheets in folder: {len(sheets)}')
print(f'data rows: {data.shape[0]}')
print(f'sheets processed: {counts}')

number of sheets in folder: 428
data rows: 86834
sheets processed: 428


In [40]:
data.head()

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
0,COVC 22771,FAM,Undetermined,NaN,2021-01-01 18:53:31
1,COVC 22771,VIC,Undetermined,NaN,2021-01-01 18:53:31
2,COVC 22771,CY5,27.861502,27.861502,2021-01-01 18:53:31
3,COVC 22772,FAM,Undetermined,NaN,2021-01-01 18:53:31
4,COVC 22772,VIC,Undetermined,NaN,2021-01-01 18:53:31


In [41]:
data.info()#.nunique()#.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86834 entries, 0 to 86833
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Sample_Name  85355 non-null  object        
 1   Reporter     85555 non-null  object        
 2   CT           85264 non-null  object        
 3   Ct_Mean      40290 non-null  float64       
 4   Dt_Run       85734 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 3.3+ MB


In [42]:
# undetermined_ids = list(data[data['CT'] == 'Undetermined']['Sample_Name'].unique())#.astype(list)

In [43]:
data2 = data[data['CT'] != 'Undetermined']

In [44]:
data3 = data2[data2['Reporter'].isin(['FAM', 'VIC', 'ABY']) == True]

In [45]:
data1 = data3[data3['Sample_Name'].isin(['PC', 'NC']) != True]

In [46]:
data4 = data1[data1.duplicated('Sample_Name', False) == True]

In [47]:
data4.head(10)

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
12,COVC 22775,FAM,30.944201,30.944201,2021-01-01 18:53:31
13,COVC 22775,VIC,33.318077,33.318077,2021-01-01 18:53:31
15,COVC 22776,FAM,27.315727,27.315727,2021-01-01 18:53:31
16,COVC 22776,VIC,30.182444,30.182444,2021-01-01 18:53:31
18,COVC 22777,FAM,27.839359,27.839359,2021-01-01 18:53:31
19,COVC 22777,VIC,30.251589,30.251589,2021-01-01 18:53:31
27,COVC 22434-PC,FAM,30.890799,30.890799,2021-01-01 18:53:31
28,COVC 22434-PC,VIC,32.892776,32.892776,2021-01-01 18:53:31
45,COVC22918,FAM,39.543823,39.543823,2021-02-01 17:26:49
46,COVC22918,VIC,40.423973,40.423973,2021-02-01 17:26:49


In [48]:
def get_upper_alpha_seq():
    letters = []
    for c in range(65, 91):
        letters.append(chr(c))
    return letters

In [49]:
def cov_rename(x):
    try:
        if 'COV' not in x and '-R' in x:
            x = x.replace('-R', '')
        if 'COV' in x and 'COVC' not in x:
            x = x.replace('COV', 'COVC')
        int(x)
        x = str(x).replace(str(x), f'COVC{str(x)}')
    except (ValueError, AttributeError, TypeError): x = str(x).replace(' ', '')
    return x

In [50]:
def cov_names(x):
    cov_id = str(x).upper().replace(' ', '').rstrip(f'#-_{get_upper_alpha_seq()}')
    if len(cov_id) == 8:
        return cov_id.replace('COVC', 'COVC0')
    elif len(cov_id) == 7:
        return cov_id.replace('COVC', 'COVC00')
    elif len(cov_id) == 6:
        return cov_id.replace('COVC', 'COVC000')
    elif len(cov_id) == 5:
        return cov_id.replace('COVC', 'COVC0000')
    return cov_id

In [51]:
data4['Reporter'].unique()

array(['FAM', 'VIC', 'ABY'], dtype=object)

In [52]:
data5 = data4.assign(Sample_Name=data4['Sample_Name'].apply(lambda x: cov_names(cov_rename(x))))

In [53]:
data5.head()

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
12,COVC22775,FAM,30.944201,30.944201,2021-01-01 18:53:31
13,COVC22775,VIC,33.318077,33.318077,2021-01-01 18:53:31
15,COVC22776,FAM,27.315727,27.315727,2021-01-01 18:53:31
16,COVC22776,VIC,30.182444,30.182444,2021-01-01 18:53:31
18,COVC22777,FAM,27.839359,27.839359,2021-01-01 18:53:31


In [54]:
(data5.sort_values(['Sample_Name', 'Reporter', 'Dt_Run'])
 .drop_duplicates(['Sample_Name', 'Reporter'], keep='last', inplace=True))

In [55]:
data5[data5['Sample_Name'].str.contains('COVC') == False]#['Dt_Run'].unique()

,Sample_Name,Reporter,CT,Ct_Mean,Dt_Run
367,CTRL1,ABY,18.343365,18.343365,2020-07-01 17:52:31
371,CTRL2,ABY,10.159125,10.159125,2020-07-01 17:52:31
1833,,FAM,37.978046,37.978046,2020-09-01 14:29:39
1834,,VIC,40.32642,40.326420,2020-09-01 14:29:39
2163,,FAM,32.339165,32.339165,2020-10-01 21:17:06
...,...,...,...,...,...
78077,SAMPLE7,VIC,23.493408,23.493408,2021-05-28 17:52:48
78079,SAMPLE8,FAM,19.822842,19.822842,2021-05-28 17:52:48
78080,SAMPLE8,VIC,21.812548,21.812548,2021-05-28 17:52:48
78094,SAMPLE13,FAM,22.972322,22.972322,2021-05-28 17:52:48


In [56]:
data6 = data5.sort_values(['Sample_Name']).groupby('Sample_Name').mean()

In [57]:
data6#.head()

,Ct_Mean
Sample_Name,
,27.688557
COVC00137,37.646282
COVC00154,35.240269
COVC00744,34.715912
COVC00762,36.049999
...,...
SEWAGE2,37.252859
STD1_2X10^4,25.153202
STD3_2X10^2,30.812906


In [58]:
df_cts = data6.reset_index()[data6.reset_index()['Sample_Name'].str.contains('COV') == True]

In [59]:
df_cts

,Sample_Name,Ct_Mean
1,COVC00137,37.646282
2,COVC00154,35.240269
3,COVC00744,34.715912
4,COVC00762,36.049999
5,COVC00767,34.074465
...,...,...
4032,COVC_8176,35.033876
4033,COVC_8177,25.184707
4034,COVC_8191,23.228168
4035,COVC_8193,26.923361


In [60]:
df_cts.to_excel('/home/douso/Documents/RunSheets/Cts/runsheet-cts.xlsx', index=False, float_format='%.1f')